## Preprocessing Step

This notebook aims to preprocess raw reservation data, making the necessary adjustments for further processing.

Importing libraries and data:

In [227]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [228]:
data = pd.read_csv('../csv/rawData.csv')

#### Let's explore the dataset:

In [229]:
data.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='object')

In [230]:
data.shape

(36275, 19)

We have 36270 instances with 18 features.

In [231]:
data.head(5)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


Let's see if there are null values:

In [232]:
data.isnull().sum()

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

Shuffling the data is a good practice to prevent any bias introduced by the order of the data, ensuring better model generalization.

In [233]:
data = data.sample(frac=1)

#### So, now that we know that there are no null values, we are going to transform the non-numerical columns.

- We can see that the column *Booking_ID* and *arrival_year* is useless as it doesn't give information. So we will drop it.

In [234]:
data.drop(['Booking_ID', 'arrival_year'], inplace=True, axis=1)
data.head(5)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36054,2,0,2,1,Meal Plan 1,0,Room_Type 1,133,6,19,Online,0,0,0,120.0,1,Not_Canceled
24590,2,0,2,0,Meal Plan 1,0,Room_Type 1,3,10,16,Online,0,0,0,120.0,0,Not_Canceled
18331,2,0,1,3,Meal Plan 1,0,Room_Type 1,100,5,19,Online,0,0,0,136.0,1,Not_Canceled
3748,2,0,2,1,Not Selected,0,Room_Type 1,10,10,29,Online,0,0,0,120.0,3,Not_Canceled
19833,3,0,0,3,Meal Plan 1,0,Room_Type 4,166,8,4,Online,0,0,0,137.7,0,Canceled


- Next one is *type_of_meal_plan*. Let's see the distribution:

In [235]:
data['type_of_meal_plan'].value_counts()

Meal Plan 1     27835
Not Selected     5130
Meal Plan 2      3305
Meal Plan 3         5
Name: type_of_meal_plan, dtype: int64

As 'Meal Plan 3' is scarcely represented, with only 5 instances, we will remove those rows to prevent biasing the analysis and ensure robust model performance.

In [236]:
data = data.loc[data['type_of_meal_plan'] != 'Meal Plan 3']
data['type_of_meal_plan'].value_counts()

Meal Plan 1     27835
Not Selected     5130
Meal Plan 2      3305
Name: type_of_meal_plan, dtype: int64

Finally, we encode this column:

In [237]:
encoder_meal = LabelEncoder()
data['type_of_meal_plan'] = encoder_meal.fit_transform(data['type_of_meal_plan'])
data['type_of_meal_plan'].value_counts()

0    27835
2     5130
1     3305
Name: type_of_meal_plan, dtype: int64

In [238]:
data.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36054,2,0,2,1,0,0,Room_Type 1,133,6,19,Online,0,0,0,120.0,1,Not_Canceled
24590,2,0,2,0,0,0,Room_Type 1,3,10,16,Online,0,0,0,120.0,0,Not_Canceled
18331,2,0,1,3,0,0,Room_Type 1,100,5,19,Online,0,0,0,136.0,1,Not_Canceled


- Now is the turn of *room_type_reserved*:

In [239]:
data['room_type_reserved'].value_counts()

Room_Type 1    28129
Room_Type 4     6056
Room_Type 6      966
Room_Type 2      692
Room_Type 5      265
Room_Type 7      155
Room_Type 3        7
Name: room_type_reserved, dtype: int64

Room Types 6, 2, 5, 7 and 3 are less represented than the others. So, again, we are going to join them.

In [240]:
data.loc[(data['room_type_reserved'] != "Room_Type 1") & (data['room_type_reserved'] != "Room_Type 4"), 'room_type_reserved'] = 'OtherRoomType'
data['room_type_reserved'].value_counts()

Room_Type 1      28129
Room_Type 4       6056
OtherRoomType     2085
Name: room_type_reserved, dtype: int64

And we encode the column.

In [241]:
encoder_room = LabelEncoder()
data['room_type_reserved'] = encoder_room.fit_transform(data['room_type_reserved'])
data['room_type_reserved'].value_counts()

1    28129
2     6056
0     2085
Name: room_type_reserved, dtype: int64

In [242]:
data.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36054,2,0,2,1,0,0,1,133,6,19,Online,0,0,0,120.0,1,Not_Canceled
24590,2,0,2,0,0,0,1,3,10,16,Online,0,0,0,120.0,0,Not_Canceled
18331,2,0,1,3,0,0,1,100,5,19,Online,0,0,0,136.0,1,Not_Canceled


- Now we go with *market_segment_type*:

In [243]:
data['market_segment_type'].value_counts()

Online           23214
Offline          10527
Corporate         2017
Complementary      387
Aviation           125
Name: market_segment_type, dtype: int64

We join 'Corporate', 'Complementary' and 'Aviation':

In [244]:
data.loc[(data['market_segment_type'] != "Online") & (data['market_segment_type'] != "Offline"), 'market_segment_type'] = 'OtherMarketSegment'
data['market_segment_type'].value_counts()

Online                23214
Offline               10527
OtherMarketSegment     2529
Name: market_segment_type, dtype: int64

Encode the column:

In [245]:
encoder_segment = LabelEncoder()
data['market_segment_type'] = encoder_segment.fit_transform(data['market_segment_type'])
data['market_segment_type'].value_counts()

1    23214
0    10527
2     2529
Name: market_segment_type, dtype: int64

In [246]:
data.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36054,2,0,2,1,0,0,1,133,6,19,1,0,0,0,120.0,1,Not_Canceled
24590,2,0,2,0,0,0,1,3,10,16,1,0,0,0,120.0,0,Not_Canceled
18331,2,0,1,3,0,0,1,100,5,19,1,0,0,0,136.0,1,Not_Canceled


#### Right now, all of the data is numerical except for the target column, the feature we want to predict, *booking_status*.

Therefore, let's encode and extract it:

In [247]:
encoder_target = LabelEncoder()
data['booking_status'] = encoder_target.fit_transform(data['booking_status'])
data['booking_status'].value_counts()

1    24386
0    11884
Name: booking_status, dtype: int64

In [248]:
data.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36054,2,0,2,1,0,0,1,133,6,19,1,0,0,0,120.0,1,1
24590,2,0,2,0,0,0,1,3,10,16,1,0,0,0,120.0,0,1
18331,2,0,1,3,0,0,1,100,5,19,1,0,0,0,136.0,1,1


In [249]:
target = data['booking_status']
target.head(3)

36054    1
24590    1
18331    1
Name: booking_status, dtype: int32

In [250]:
data.drop(['booking_status'], inplace=True, axis=1)
data.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
36054,2,0,2,1,0,0,1,133,6,19,1,0,0,0,120.0,1
24590,2,0,2,0,0,0,1,3,10,16,1,0,0,0,120.0,0
18331,2,0,1,3,0,0,1,100,5,19,1,0,0,0,136.0,1


#### At this moment, the only step left is to normalize the data.

In [251]:
minmax = MinMaxScaler(feature_range=(-1, 1))
data = minmax.fit_transform(data)
data = pd.DataFrame(data)
data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,-1.0,-0.428571,-0.882353,-1.0,-1.0,0.0,-0.399549,-0.090909,0.2,0.0,-1.0,-1.0,-1.0,-0.555556,-0.6
1,0.0,-1.0,-0.428571,-1.000000,-1.0,-1.0,0.0,-0.986456,0.636364,0.0,0.0,-1.0,-1.0,-1.0,-0.555556,-1.0
2,0.0,-1.0,-0.714286,-0.647059,-1.0,-1.0,0.0,-0.548533,-0.272727,0.2,0.0,-1.0,-1.0,-1.0,-0.496296,-0.6


As we can see, the data has been normalized within the range (-1, 1). Dimensions:

In [252]:
data.shape

(36270, 16)

Our dataset is ready. It has, finally, 36270 instances with 16 features.

#### Now, we can export the data and build the neural network.

In [253]:
data.to_csv('../csv/preprocessedData.csv', index=False)
target.to_csv('../csv/target.csv', index=False)